# Aim
Create refrence archor images for age prediction using Twin Network.

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import cv2
import glob
import json
import matplotlib as mpl
import matplotlib.pyplot as plt
import numpy as np
import os
import pandas as pd
import pathlib
import shutil
import tensorflow as tf
import tensorflow_addons as tfa
import tensorflow_io as tfio
from tensorflow.keras import applications, layers, models

import os
import random

os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'
os.environ['TF_FORCE_GPU_ALLOW_GROWTH'] = 'true'


from twinnet_tools.tnconfig import ProjectConfig

config = ProjectConfig("twinnet_config")
dir_root_scripts = config.json["dir_scripts"]


import glob
import matplotlib as mpl
import pandas as pd
from pathlib import Path
import sys

sys.path.append(dir_root_scripts)
from twinnet_tools.tngeneral import TNToolsGeneral
from twinnet_tools.tninference import TNToolsEmbeddings
from twinnet_tools.tninference import TNToolsSimilarities
from twinnet_tools.tnmodel import TNToolsNetwork
from twinnet_tools.tnplot import TNToolsPlot

# Import tools

tools_general = TNToolsGeneral()
tools_embeddings = TNToolsEmbeddings(size_img=224,
                                     size_img_min=250)
tools_model = TNToolsNetwork()
tools_similarities = TNToolsSimilarities()
tools_plot = TNToolsPlot()

ProjectConfig: C:\TwinNet\TwinNet-main\code\Scripts\twinnet_config\Windows\config.json


In [3]:
new_rc_params = {'text.usetex': False,
                'svg.fonttype': 'none',
                'lines.linewidth': 1}

mpl.rcParams.update(new_rc_params)

random.seed(1) # for reproducibility 

# 1. Paths

In [4]:
dir_src_imgs = "C:/TwinNet/Medaka/segmented/trainData/CombinedData/AllEmbryos/"
dir_dst = "C:/TwinNet/Medaka//images_jsons_reference_medaka/"
relative_path = "./images_jsons_reference_medaka/images_reference/"
Number_anchors = 15

Create output directories

In [5]:
def create_folder(subfolder_path, folder_name):
    # Join the subfolder path and the folder name
    folder_path = os.path.join(subfolder_path, folder_name)

    try:
        # Create the folder
        os.makedirs(folder_path)
        print(f"Folder '{folder_name}' created successfully in '{subfolder_path}'!")
    except FileExistsError:
        print(f"Folder '{folder_name}' already exists in '{subfolder_path}'!")
        
def get_time_stamp(file_name):
    parts = file_name.split('--')
    time_stamp = parts[2].split('LO')[1]
    return time_stamp     

def get_images_by_subfolder(folder):
 
    embryoIds = sorted([folder for folder in os.listdir(dir_src_imgs) if os.path.isdir(os.path.join(dir_src_imgs, folder))])
    image_lists = []
 
    for subfolder in embryoIds:
        subfolder_path = os.path.join(folder, subfolder)
        tif_files = sorted([file for file in os.listdir(subfolder_path) if file.endswith('.tif')])
        image_lists.append(tif_files)

        
    return image_lists


def get_images_paths_by_subfolder(folder):
    
    embryoIds = sorted([folder for folder in os.listdir(dir_src_imgs) if os.path.isdir(os.path.join(dir_src_imgs, folder))])
    image_lists = []
 
    for subfolder in embryoIds:
        subfolder_path = os.path.join(folder, subfolder)
        tif_files = sorted([os.path.join(subfolder_path, file) for file in os.listdir(subfolder_path) if file.endswith('.tif')])
        image_lists.append(tif_files)

    return image_lists
    
    


def save_json(data, output_file):
    """Save json file."""
    with open(output_file, 'w') as file:
        json.dump(data, file, separators=(',', ':'))


In [6]:
# Cerate output directories

create_folder(dir_dst, 'images_reference')
images_reference_path = os.path.join(dir_dst, 'images_reference')

path_out = [None]*Number_anchors
# Iterate over the range from 1 to Number_anchors
for i in range(1, Number_anchors+1):
    i_padded = str(i).zfill(len(str(Number_anchors)))
    imgs_anchor_name = 'imgs_anchor_{}'.format(i_padded)
    create_folder(images_reference_path, imgs_anchor_name)
    path_out[i-1] = os.path.join(images_reference_path, imgs_anchor_name)
                               

Folder 'images_reference' created successfully in 'C:/TwinNet/Medaka//images_jsons_reference_medaka/'!
Folder 'imgs_anchor_01' created successfully in 'C:/TwinNet/Medaka//images_jsons_reference_medaka/images_reference'!
Folder 'imgs_anchor_02' created successfully in 'C:/TwinNet/Medaka//images_jsons_reference_medaka/images_reference'!
Folder 'imgs_anchor_03' created successfully in 'C:/TwinNet/Medaka//images_jsons_reference_medaka/images_reference'!
Folder 'imgs_anchor_04' created successfully in 'C:/TwinNet/Medaka//images_jsons_reference_medaka/images_reference'!
Folder 'imgs_anchor_05' created successfully in 'C:/TwinNet/Medaka//images_jsons_reference_medaka/images_reference'!
Folder 'imgs_anchor_06' created successfully in 'C:/TwinNet/Medaka//images_jsons_reference_medaka/images_reference'!
Folder 'imgs_anchor_07' created successfully in 'C:/TwinNet/Medaka//images_jsons_reference_medaka/images_reference'!
Folder 'imgs_anchor_08' created successfully in 'C:/TwinNet/Medaka//images_jso

# 2. Choose rand images from all avaiable embryos

In [7]:
# Get avaible embryos and images
embryoIds = sorted([folder for folder in os.listdir(dir_src_imgs) if os.path.isdir(os.path.join(dir_src_imgs, folder))])
images_by_embryo = get_images_by_subfolder(dir_src_imgs)
images_paths_by_embryo = get_images_paths_by_subfolder(dir_src_imgs)
time_stamps = [get_time_stamp(file_name) for file_name in images_by_embryo[0]]   

for i, image_list in enumerate(images_by_embryo, 1):
    print(f"Subfolder {embryoIds[i-1]} contains : {len(image_list)} images")
    

# Get images N-plets and paths

# Create lists to store the paths
N_embryos = len(embryoIds) 
N_times = len(time_stamps)

paths_anchor = [None] * Number_anchors

for i in range(1, Number_anchors+1):
    i_padded = str(i).zfill(len(str(Number_anchors)))
    imgs_anchor_name = 'imgs_anchor_{}'.format(i_padded) + '/'
    paths_anchor[i-1] = [ os.path.join(relative_path, imgs_anchor_name)] * N_times



Subfolder E012 contains : 720 images
Subfolder E013 contains : 720 images
Subfolder E015 contains : 720 images
Subfolder E016 contains : 720 images
Subfolder E017 contains : 720 images
Subfolder E019 contains : 720 images
Subfolder E020 contains : 720 images
Subfolder E021 contains : 720 images
Subfolder E022 contains : 720 images
Subfolder E023 contains : 720 images
Subfolder E024 contains : 720 images
Subfolder E025 contains : 720 images
Subfolder E026 contains : 720 images
Subfolder E028 contains : 720 images
Subfolder E029 contains : 720 images
Subfolder E030 contains : 720 images
Subfolder E034 contains : 720 images
Subfolder E035 contains : 720 images
Subfolder E038 contains : 720 images
Subfolder E039 contains : 720 images
Subfolder E045 contains : 720 images
Subfolder E046 contains : 720 images
Subfolder E048 contains : 720 images
Subfolder E049 contains : 720 images
Subfolder E056 contains : 720 images
Subfolder E057 contains : 720 images
Subfolder E058 contains : 720 images
S

In [8]:
# get a triplet for each time point
for counter, time_stamp in enumerate(time_stamps, 1):
    random_Nplet = []
    while len(random_Nplet) < Number_anchors:
        num = random.randint(0, N_embryos-1)
        if num not in random_Nplet:
            random_Nplet.append(num)
    print(f"Counter: {counter}, Time Stamp: {time_stamp}, Random Nplet: {random_Nplet}")
    
    for i in range(0, Number_anchors):
        # check file consistency
        imgName = images_by_embryo[random_Nplet[i]][counter-1]
        if get_time_stamp(imgName) == time_stamp:
            # Store relative paths
            paths_anchor[i][counter-1] = os.path.join(paths_anchor[i][counter-1], imgName)
            #Copy images
            img_path = images_paths_by_embryo[random_Nplet[i]][counter-1]
            shutil.copyfile(img_path, os.path.join(path_out[i], imgName))      
        else:
            print(f"ERROR '{imgName} ' does not match time stamp '{time_stamp}'!")
            
    

# Save json files

for i in range(1, Number_anchors+1):
    i_padded = str(i).zfill(len(str(Number_anchors)))
    outname = 'paths_anchor_{}'.format(i_padded)+'.json'
    save_json(paths_anchor[i-1], os.path.join(dir_dst, outname))
       

Counter: 1, Time Stamp: 001, Random Nplet: [34, 145, 16, 65, 30, 126, 115, 120, 97, 53, 24, 124, 7, 99, 110]
Counter: 2, Time Stamp: 002, Random Nplet: [0, 114, 68, 58, 26, 81, 7, 5, 6, 138, 2, 97, 55, 108, 135]
Counter: 3, Time Stamp: 003, Random Nplet: [56, 112, 126, 141, 59, 88, 117, 74, 5, 106, 142, 25, 47, 75, 30]
Counter: 4, Time Stamp: 004, Random Nplet: [85, 128, 108, 129, 48, 77, 72, 127, 100, 8, 122, 62, 103, 106, 44]
Counter: 5, Time Stamp: 005, Random Nplet: [93, 140, 95, 22, 112, 130, 27, 41, 133, 100, 94, 125, 7, 120, 11]
Counter: 6, Time Stamp: 006, Random Nplet: [78, 100, 43, 128, 58, 3, 51, 138, 140, 59, 103, 131, 88, 90, 117]
Counter: 7, Time Stamp: 007, Random Nplet: [68, 140, 1, 98, 131, 33, 132, 143, 52, 109, 14, 123, 93, 145, 141]
Counter: 8, Time Stamp: 008, Random Nplet: [51, 129, 105, 124, 91, 106, 88, 0, 137, 138, 84, 117, 7, 58, 45]
Counter: 9, Time Stamp: 009, Random Nplet: [140, 46, 23, 141, 65, 8, 18, 21, 4, 115, 3, 71, 63, 68, 28]
Counter: 10, Time Stamp:

Counter: 79, Time Stamp: 079, Random Nplet: [103, 23, 135, 121, 144, 107, 137, 100, 77, 56, 140, 34, 13, 130, 28]
Counter: 80, Time Stamp: 080, Random Nplet: [44, 61, 55, 111, 70, 139, 5, 64, 137, 69, 135, 67, 121, 32, 103]
Counter: 81, Time Stamp: 081, Random Nplet: [26, 95, 17, 139, 92, 142, 129, 7, 78, 114, 33, 39, 19, 36, 55]
Counter: 82, Time Stamp: 082, Random Nplet: [123, 85, 93, 74, 40, 39, 97, 112, 103, 30, 37, 69, 75, 2, 137]
Counter: 83, Time Stamp: 083, Random Nplet: [2, 33, 97, 143, 25, 117, 7, 110, 108, 70, 94, 104, 103, 118, 13]
Counter: 84, Time Stamp: 084, Random Nplet: [25, 120, 9, 0, 10, 28, 35, 135, 130, 91, 141, 69, 145, 121, 62]
Counter: 85, Time Stamp: 085, Random Nplet: [61, 27, 143, 91, 40, 29, 10, 80, 108, 88, 64, 14, 111, 106, 96]
Counter: 86, Time Stamp: 086, Random Nplet: [91, 75, 87, 112, 60, 132, 36, 14, 29, 131, 44, 139, 124, 31, 5]
Counter: 87, Time Stamp: 087, Random Nplet: [122, 53, 98, 44, 101, 58, 25, 63, 85, 84, 62, 118, 120, 94, 126]
Counter: 88, 

Counter: 155, Time Stamp: 155, Random Nplet: [104, 39, 28, 136, 45, 87, 36, 96, 111, 83, 138, 133, 70, 53, 49]
Counter: 156, Time Stamp: 156, Random Nplet: [40, 42, 137, 41, 37, 30, 113, 133, 33, 110, 34, 85, 81, 35, 5]
Counter: 157, Time Stamp: 157, Random Nplet: [91, 44, 57, 60, 127, 125, 8, 22, 34, 136, 120, 144, 36, 53, 92]
Counter: 158, Time Stamp: 158, Random Nplet: [35, 71, 89, 16, 98, 121, 7, 135, 118, 50, 61, 52, 1, 77, 10]
Counter: 159, Time Stamp: 159, Random Nplet: [68, 132, 48, 18, 27, 28, 102, 84, 26, 114, 133, 123, 71, 36, 110]
Counter: 160, Time Stamp: 160, Random Nplet: [95, 89, 98, 105, 111, 94, 140, 52, 50, 16, 37, 60, 61, 5, 100]
Counter: 161, Time Stamp: 161, Random Nplet: [116, 112, 145, 24, 13, 44, 134, 1, 11, 110, 71, 106, 33, 60, 95]
Counter: 162, Time Stamp: 162, Random Nplet: [106, 87, 12, 129, 116, 33, 133, 93, 15, 89, 30, 62, 31, 111, 38]
Counter: 163, Time Stamp: 163, Random Nplet: [4, 93, 33, 38, 73, 6, 120, 123, 17, 110, 23, 139, 128, 24, 32]
Counter: 16

Counter: 230, Time Stamp: 230, Random Nplet: [124, 52, 130, 22, 86, 103, 45, 60, 132, 18, 107, 101, 55, 65, 0]
Counter: 231, Time Stamp: 231, Random Nplet: [72, 8, 66, 21, 46, 64, 114, 111, 78, 25, 76, 13, 122, 44, 65]
Counter: 232, Time Stamp: 232, Random Nplet: [140, 53, 32, 10, 102, 141, 2, 145, 128, 76, 0, 97, 87, 24, 65]
Counter: 233, Time Stamp: 233, Random Nplet: [41, 51, 18, 44, 96, 135, 145, 5, 56, 103, 4, 1, 106, 13, 101]
Counter: 234, Time Stamp: 234, Random Nplet: [106, 48, 40, 56, 23, 115, 138, 139, 85, 64, 49, 129, 65, 99, 63]
Counter: 235, Time Stamp: 235, Random Nplet: [74, 65, 37, 68, 92, 70, 128, 56, 48, 137, 5, 26, 55, 43, 82]
Counter: 236, Time Stamp: 236, Random Nplet: [56, 41, 8, 99, 67, 65, 54, 66, 96, 10, 39, 127, 111, 77, 93]
Counter: 237, Time Stamp: 237, Random Nplet: [103, 91, 50, 73, 71, 67, 123, 39, 36, 99, 15, 18, 66, 19, 126]
Counter: 238, Time Stamp: 238, Random Nplet: [53, 116, 78, 10, 68, 86, 1, 126, 110, 108, 93, 124, 49, 100, 74]
Counter: 239, Time 

Counter: 317, Time Stamp: 317, Random Nplet: [27, 85, 71, 36, 67, 11, 24, 42, 9, 1, 83, 86, 10, 41, 107]
Counter: 318, Time Stamp: 318, Random Nplet: [112, 4, 110, 140, 144, 102, 31, 101, 14, 7, 34, 61, 129, 124, 99]
Counter: 319, Time Stamp: 319, Random Nplet: [80, 32, 73, 35, 145, 144, 141, 36, 37, 60, 50, 7, 54, 125, 143]
Counter: 320, Time Stamp: 320, Random Nplet: [117, 93, 120, 109, 135, 0, 107, 61, 97, 68, 6, 134, 82, 27, 131]
Counter: 321, Time Stamp: 321, Random Nplet: [62, 109, 65, 33, 133, 119, 145, 140, 94, 64, 31, 23, 89, 103, 113]
Counter: 322, Time Stamp: 322, Random Nplet: [93, 107, 129, 112, 127, 96, 137, 7, 12, 124, 82, 59, 26, 5, 89]
Counter: 323, Time Stamp: 323, Random Nplet: [19, 45, 121, 31, 59, 37, 117, 42, 71, 93, 47, 11, 74, 113, 139]
Counter: 324, Time Stamp: 324, Random Nplet: [141, 116, 72, 62, 2, 77, 120, 38, 50, 46, 78, 18, 65, 43, 100]
Counter: 325, Time Stamp: 325, Random Nplet: [77, 46, 116, 101, 23, 91, 59, 0, 138, 42, 32, 72, 98, 38, 86]
Counter: 326

Counter: 400, Time Stamp: 400, Random Nplet: [39, 128, 144, 114, 17, 121, 65, 104, 138, 142, 31, 98, 30, 68, 96]
Counter: 401, Time Stamp: 401, Random Nplet: [8, 23, 139, 88, 19, 100, 120, 122, 40, 102, 10, 66, 101, 98, 115]
Counter: 402, Time Stamp: 402, Random Nplet: [3, 62, 137, 33, 18, 127, 103, 53, 27, 118, 50, 111, 6, 71, 5]
Counter: 403, Time Stamp: 403, Random Nplet: [11, 124, 107, 33, 45, 48, 129, 96, 72, 20, 140, 10, 105, 37, 119]
Counter: 404, Time Stamp: 404, Random Nplet: [40, 76, 10, 74, 8, 134, 83, 75, 124, 22, 41, 98, 126, 67, 114]
Counter: 405, Time Stamp: 405, Random Nplet: [5, 107, 8, 75, 12, 88, 51, 87, 46, 62, 35, 23, 10, 24, 39]
Counter: 406, Time Stamp: 406, Random Nplet: [78, 80, 126, 121, 5, 98, 139, 97, 60, 134, 36, 140, 26, 106, 141]
Counter: 407, Time Stamp: 407, Random Nplet: [98, 57, 142, 2, 143, 80, 104, 99, 55, 33, 111, 45, 28, 145, 76]
Counter: 408, Time Stamp: 408, Random Nplet: [44, 90, 120, 127, 134, 41, 68, 61, 67, 18, 130, 83, 62, 142, 56]
Counter:

Counter: 487, Time Stamp: 487, Random Nplet: [78, 39, 123, 13, 119, 106, 22, 4, 2, 11, 7, 55, 94, 52, 115]
Counter: 488, Time Stamp: 488, Random Nplet: [11, 12, 56, 76, 51, 143, 45, 49, 50, 95, 136, 55, 34, 145, 25]
Counter: 489, Time Stamp: 489, Random Nplet: [80, 41, 139, 118, 127, 5, 138, 122, 17, 3, 116, 81, 91, 88, 105]
Counter: 490, Time Stamp: 490, Random Nplet: [36, 31, 27, 118, 83, 42, 17, 98, 5, 129, 113, 0, 34, 75, 71]
Counter: 491, Time Stamp: 491, Random Nplet: [77, 15, 121, 145, 38, 0, 127, 105, 9, 62, 29, 99, 137, 134, 7]
Counter: 492, Time Stamp: 492, Random Nplet: [78, 1, 5, 61, 135, 98, 71, 93, 54, 109, 115, 128, 12, 33, 138]
Counter: 493, Time Stamp: 493, Random Nplet: [69, 8, 24, 101, 12, 126, 125, 38, 28, 64, 46, 15, 27, 136, 118]
Counter: 494, Time Stamp: 494, Random Nplet: [18, 46, 86, 119, 136, 40, 124, 95, 65, 76, 123, 44, 77, 133, 7]
Counter: 495, Time Stamp: 495, Random Nplet: [45, 62, 26, 99, 19, 31, 75, 25, 64, 56, 44, 92, 27, 115, 82]
Counter: 496, Time St

Counter: 566, Time Stamp: 566, Random Nplet: [21, 130, 3, 49, 134, 92, 112, 29, 48, 4, 6, 142, 26, 78, 5]
Counter: 567, Time Stamp: 567, Random Nplet: [106, 140, 44, 138, 66, 45, 11, 5, 144, 48, 30, 92, 12, 79, 28]
Counter: 568, Time Stamp: 568, Random Nplet: [42, 55, 0, 12, 59, 43, 107, 36, 1, 130, 119, 125, 8, 37, 127]
Counter: 569, Time Stamp: 569, Random Nplet: [65, 118, 143, 91, 86, 106, 60, 95, 119, 122, 126, 1, 48, 85, 49]
Counter: 570, Time Stamp: 570, Random Nplet: [25, 85, 119, 11, 47, 66, 140, 62, 114, 110, 32, 44, 10, 130, 145]
Counter: 571, Time Stamp: 571, Random Nplet: [80, 123, 76, 141, 65, 71, 135, 128, 96, 62, 133, 57, 87, 78, 70]
Counter: 572, Time Stamp: 572, Random Nplet: [81, 97, 66, 62, 21, 126, 42, 95, 140, 40, 138, 20, 113, 122, 93]
Counter: 573, Time Stamp: 573, Random Nplet: [81, 19, 137, 132, 55, 56, 37, 90, 87, 138, 131, 139, 43, 42, 49]
Counter: 574, Time Stamp: 574, Random Nplet: [52, 138, 8, 145, 106, 48, 117, 105, 84, 7, 80, 45, 32, 59, 107]
Counter: 57

Counter: 646, Time Stamp: 646, Random Nplet: [24, 116, 82, 66, 16, 57, 76, 104, 77, 42, 47, 74, 63, 44, 36]
Counter: 647, Time Stamp: 647, Random Nplet: [57, 119, 122, 94, 2, 87, 101, 12, 61, 80, 121, 6, 126, 92, 132]
Counter: 648, Time Stamp: 648, Random Nplet: [19, 16, 36, 13, 112, 23, 95, 67, 61, 39, 65, 132, 92, 101, 125]
Counter: 649, Time Stamp: 649, Random Nplet: [45, 127, 84, 137, 106, 50, 70, 102, 112, 143, 22, 1, 54, 92, 21]
Counter: 650, Time Stamp: 650, Random Nplet: [8, 46, 125, 126, 99, 58, 108, 92, 113, 60, 5, 132, 44, 119, 47]
Counter: 651, Time Stamp: 651, Random Nplet: [58, 31, 12, 103, 132, 10, 54, 81, 33, 51, 36, 145, 99, 19, 37]
Counter: 652, Time Stamp: 652, Random Nplet: [113, 66, 123, 95, 138, 142, 63, 10, 46, 90, 67, 140, 32, 130, 82]
Counter: 653, Time Stamp: 653, Random Nplet: [17, 133, 126, 37, 62, 60, 55, 90, 58, 84, 31, 82, 34, 69, 36]
Counter: 654, Time Stamp: 654, Random Nplet: [112, 89, 69, 102, 11, 117, 130, 67, 111, 82, 8, 92, 76, 57, 12]
Counter: 655